<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/10038/logos/header.png?t=2018-06-21-23-13-17">

In [1]:
# Load required packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import json # to convert json in df
from pandas.io.json import json_normalize # to normalize the json file

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

There are 13 columns out of which 'device', 'geoNetwork', 'totals', 'trafficSource' are in Json format 
So need to reload the data with correct format Lets create function

In [2]:
def load_df1(csv_path='../DATA/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../DATA"))

['.DS_Store', 'test_v2.csv', 'sample_submission_v2.csv', 'train_v2.csv', 'train_v2.csv.zip', 'test_v2.csv.zip', 'sample_submission_v2.csv.zip']


In [ ]:
# Lets read train and test data

In [4]:
%%time
df_train = load_df1()
df_test = load_df1("../DATA/test_v2.csv")

Loaded train_v2.csv. Shape: (1708337, 60)
Loaded test_v2.csv. Shape: (401589, 59)
CPU times: user 8min 55s, sys: 4min 11s, total: 13min 6s
Wall time: 18min 21s


In [5]:
df_train.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,1,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windj

Lets look at unique values in each columns

In [9]:
df_train['channelGrouping'].unique()

array(['Organic Search', 'Referral', 'Direct', 'Paid Search', 'Display',
       'Affiliates', 'Social', '(Other)'], dtype=object)

In [10]:
df_train['customDimensions'].unique()

array(["[{'index': '4', 'value': 'EMEA'}]",
       "[{'index': '4', 'value': 'North America'}]",
       "[{'index': '4', 'value': 'Central America'}]", '[]',
       "[{'index': '4', 'value': 'APAC'}]",
       "[{'index': '4', 'value': 'South America'}]"], dtype=object)

In [11]:
for column in df_train.columns:
    print("for column ",column, "unique values are")
    print(df_train[column].unique())

for column  channelGrouping unique values are
['Organic Search' 'Referral' 'Direct' 'Paid Search' 'Display' 'Affiliates'
 'Social' '(Other)']
for column  customDimensions unique values are
["[{'index': '4', 'value': 'EMEA'}]"
 "[{'index': '4', 'value': 'North America'}]"
 "[{'index': '4', 'value': 'Central America'}]" '[]'
 "[{'index': '4', 'value': 'APAC'}]"
 "[{'index': '4', 'value': 'South America'}]"]
for column  date unique values are
[20171016 20160902 20171130 20170126 20170623 20170312 20170203 20180415
 20171110 20160811 20170613 20170113 20171111 20161118 20161110 20161112
 20170621 20180117 20180103 20161123 20180120 20180307 20180429 20170501
 20170611 20170630 20161213 20180211 20180221 20171030 20170519 20180316
 20170111 20170624 20161014 20170914 20171006 20170529 20180203 20170515
 20170420 20160831 20171007 20170801 20170228 20161209 20160917 20180413
 20170617 20180204 20161216 20160820 20170616 20171115 20180119 20180402
 20160822 20180107 20161129 20170318 20171025

['Not Socially Engaged']
for column  visitId unique values are
[1508198450 1508176307 1508201613 ... 1483526434 1483592857 1483574474]
for column  visitNumber unique values are
[  1   6   2  10   3   4  14  11   7  15  17   5  13  16   8  34  21  18
  28   9  12  29 138  36  30 102  20  54  33  57  19  45  27  39  43  25
  26  23  31  99 100 101  46  24  22  56  83  38  84  42  89 136  85 105
  48  93  50  64  94  70  98 149  72  69  35 124 178  59 317  51 160  52
  63  47  49  88 140  91  92  41  74 309  61  32 108 195 162 235  90  86
 236 158  82 159 148 125  55  66 401  65  40  60  58 137 126 112 155  96
  73  78  97  87  76  77 304 107 106 389  68 132 266 267 221 143 104 142
 245 174 121 175 191 190 189 204  37 206 156 157  62 110 109 184 183 169
 163 150 147 113 164  44 194 193 254  71  53 145 144 146  80 253 222 223
  81 128 192 129 161 152 324 134 135 130  95  75 103 315 154 153 283 117
 181 272 273 118 208 122 287 133 396 115 114 131 202 232 120 262 263 295
 297 298 219 296 141

['not available in demo dataset' 'California' 'England' 'Mexico City'
 'Nevada' 'Brussels' 'Tokyo' '(not set)' 'County Dublin' 'Maharashtra'
 'Istanbul' 'Ontario' 'Telangana' 'Pennsylvania' 'Michigan'
 'Massachusetts' 'British Columbia' 'Madhya Pradesh' 'Quebec'
 'New South Wales' 'Jakarta' 'New York' 'State of Sao Paulo' 'Washington'
 'District of Columbia' 'Chiayi County' 'Delhi' 'Karnataka' 'Bangkok'
 'Aragon' 'Zurich' 'Masovian Voivodeship' 'Texas' 'Georgia' 'Illinois'
 'Tamil Nadu' 'Sindh' 'Lombardy' 'Federal Territory of Kuala Lumpur'
 'Saint Petersburg' 'Tennessee' 'Hanoi' 'Taipei City' 'Madrid' 'Berlin'
 'Ho Chi Minh City' 'Victoria' 'Seoul' 'Ile-de-France' 'Lisbon' 'Bogota'
 'New Taipei City' 'Stockholm County' 'Western Province' 'Lagos'
 'Riyadh Province' 'Dubai' 'Colorado' 'Buenos Aires' 'Lima Region'
 'Santiago Metropolitan Region' 'Dublin City' 'North Holland' 'Virginia'
 'Community of Madrid' 'West Bengal' 'Catalonia' 'State of Rio de Janeiro'
 'Queensland' 'Moscow' 'Izmi

['not available in demo dataset' 'Cupertino' 'San Francisco' 'London'
 'Mexico City' '(not set)' 'Ixelles' 'Minato' 'Los Angeles' 'Brussels'
 'Dublin' 'Mumbai' 'Mountain View' 'Istanbul' 'Toronto' 'Hyderabad'
 'Pittsburgh' 'Sunnyvale' 'Ann Arbor' 'Boston' 'Vancouver' 'Indore'
 'Montreal' 'Hong Kong' 'Sydney' 'Jakarta' 'Singapore' 'New York'
 'Sao Paulo' 'Shinjuku' 'Kirkland' 'Washington' 'Chiayi City' 'New Delhi'
 'Bengaluru' 'Bangkok' 'Zaragoza' 'Zurich' 'Warsaw' 'San Antonio'
 'Atlanta' 'Chicago' 'San Jose' 'Dallas' 'Philadelphia' 'Santa Clara'
 'Palo Alto' 'Chennai' 'Karachi' 'Irvine' 'Milan' 'San Diego' 'Cambridge'
 'Kuala Lumpur' 'Saint Petersburg' 'Austin' 'San Bruno' 'Barrie'
 'San Mateo' 'Seattle' 'Nashville' 'Hanoi' 'Madrid' 'Berlin'
 'Ho Chi Minh City' 'Melbourne' 'Houston' 'Seoul' 'Paris' 'Lisbon'
 'Bogota' 'Stockholm' 'Colombo' 'Lagos' 'Leeds' 'Riyadh' 'Dubai' 'Boulder'
 'Fremont' 'Buenos Aires' 'La Victoria' 'Pune' 'Santiago' 'Amsterdam'
 'Salem' 'Kolkata' 'Barcelona' 'Rio

['(not set)' 'windjammercable.net' 'unknown.unknown' ... 'budnet.eu'
 'lumenis.com' 'xpressgt.co.za']
for column  geoNetwork.latitude unique values are
['not available in demo dataset']
for column  geoNetwork.longitude unique values are
['not available in demo dataset']
for column  geoNetwork.networkLocation unique values are
['not available in demo dataset']
for column  totals.visits unique values are
['1']
for column  totals.hits unique values are
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '43' '45' '46'
 '47' '50' '52' '56' '58' '62' '66' '69' '74' '75' '85' '90' '93' '94'
 '138' '165' '42' '44' '48' '49' '51' '54' '55' '57' '65' '67' '72' '79'
 '82' '83' '92' '103' '109' '134' '143' '144' '53' '61' '70' '106' '137'
 '73' '112' '189' '63' '81' '91' '102' '161' '86' '99' '80' '89' '100'
 '71' '76' '77' '95' '229' '64' '154' '78' '

['organic' 'referral' '(none)' 'cpc' 'affiliate' 'cpm' '(not set)']
for column  trafficSource.keyword unique values are
['water bottle' nan '(not provided)' ... 'Mug pic' 'shorter google'
 'google shirts for sale']
for column  trafficSource.adwordsClickInfo.criteriaParameters unique values are
['not available in demo dataset']
for column  trafficSource.referralPath unique values are
[nan '/a/google.com/transportation/mtv-services/bikes/bike2workmay2016'
 '/offer/2145' ... '/mail/mu/mp/118/' '/BB1QfReObs' '/mail/mu/mp/509/']
for column  trafficSource.isTrueDirect unique values are
[nan True]
for column  trafficSource.adContent unique values are
[nan 'Placement Accessories 300 x 250' 'Google Merchandise Store'
 'Bags 300x250' 'Display Ad created 3/11/14' 'Placement Office 300x250'
 'Placement Drinkware 300x250' 'Office Placement 160x600' 'Bags 336x280'
 'test_tyler_hr_merchant' 'Drinkware 336x280'
 'Official Google Merchandise' 'First Full Auto Template Test Ad'
 'Full auto ad IMAGE ONLY

Lets look at unique count for some of categories

In [13]:
df_train.groupby('channelGrouping')['fullVisitorId'].nunique()

channelGrouping
(Other)              103
Affiliates         26507
Direct            208054
Display            35359
Organic Search    600860
Paid Search        33591
Referral          130226
Social            330209
Name: fullVisitorId, dtype: int64

In [16]:
# Lets look at unique Visitor IDs in Training Datase
print("Number of unique visitors in train set : ",df_train.fullVisitorId.nunique(), " out of rows : ",df_train.shape[0])
print("% of unique visitors in train set:", df_train.fullVisitorId.nunique()/df_train.shape[0])

Number of unique visitors in train set :  1323730  out of rows :  1708337
% of unique visitors in train set: 0.774864678339227
